In [1]:
!pip install spuco

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 3.1 MB/s eta 0:00:00


In [11]:
import spuco
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
from torchvision.transforms import ToPILImage
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import SGD
import torchvision.transforms as T
from torch.utils.data import DataLoader

In [3]:
device = torch.device("cuda:7")

In [37]:
# Subclass SpuCoMNIST to modify __getitem__ for handling batches
class CustomSpuCoMNIST(SpuCoMNIST):
    def __getitem__(self, index):
        """
        Modify __getitem__ to handle both individual indices and batches of indices (lists).
        """
        if isinstance(index, list):
            # If index is a list, return a batch of data
            X_batch = [self.data.X[i] for i in index]
            labels_batch = [self.data.labels[i] for i in index]

            if self.transform is not None:
                X_batch = [self.transform(x) for x in X_batch]

            # Convert lists to tensors or any required format
            X_batch = torch.stack(X_batch)  # Stack into a single tensor
            labels_batch = torch.tensor(labels_batch)

            return X_batch, labels_batch
        else:
            # Handle the individual case
            if self.transform is None:
                return self.data.X[index], self.data.labels[index]
            else:
                return self.transform(self.data.X[index]), self.data.labels[index]

In [38]:
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

# Initialize SpuCoMNIST dataset with required arguments

trainset = CustomSpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = CustomSpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

100%|██████████| 10000/10000 [00:01<00:00, 5084.18it/s]


In [39]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [40]:
train_loader = DataLoader(trainset, batch_size=32, shuffle=True)
test_loader = DataLoader(testset, batch_size=32, shuffle=False)

In [41]:
# Define a simple model (e.g., CNN)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.fc1 = nn.Linear(32 * 26 * 26, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = x.view(-1, 32 * 26 * 26)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

model = SimpleCNN()

In [42]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01)

In [43]:
def train_model(model, train_loader, optimizer, criterion, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

train_model(model, train_loader, optimizer, criterion)

Epoch 1, Loss: 0.06844601257959211
Epoch 2, Loss: 0.041018662806091545
Epoch 3, Loss: 0.03682998588962045
Epoch 4, Loss: 0.03400611346537901
Epoch 5, Loss: 0.03174879785673972


In [44]:
# Check if a GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [45]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:16,  1.42it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:13,  1.69it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:11,  1.85it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:13<00:00,  1.60it/s]

Group (4, 3) Accuracy: 0.0


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:13<00:00,  1.82it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.0,
 (1, 0): 0.0,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 0.0,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.0,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 0.2518891687657431,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 0.0,
 (4, 4): 100.0}

In [46]:
# Cluster inputs based on output from ERM
from sklearn.cluster import KMeans
import numpy as np

def get_outputs(model, loader):
    model.eval()
    outputs_list = []
    labels_list = []
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            outputs_list.append(outputs)
            labels_list.append(labels)
    return torch.cat(outputs_list), torch.cat(labels_list)

train_outputs, train_labels = get_outputs(model, train_loader)


In [49]:
# Retrain using Group-Balancing

def balanced_dataloader(dataset, clusters, batch_size=64):
    # Create a list of indices for each cluster
    cluster_indices = [np.where(clusters == i)[0] for i in range(max(clusters) + 1)]

    # Create a balanced DataLoader using the BalancedBatchSampler
    balanced_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=BalancedBatchSampler(cluster_indices),  # Use the custom sampler
    )
    return balanced_loader

class BalancedBatchSampler(torch.utils.data.Sampler):
    def __init__(self, clusters_indices):
        self.indices = clusters_indices

    def __iter__(self):
        # Yield a list of indices for each batch
        for i in range(len(self.indices[0])):  # Assuming each cluster has the same size
            batch = []
            for cluster in self.indices:
                # Append one random sample from each cluster to the batch
                batch.append(np.random.choice(cluster))
            # Yield batch as a list of integers (flattening it)
            yield from batch # Changed this line

    def __len__(self):
        # Return the number of batches, based on the smallest cluster
        return len(self.indices[0])  # Assuming all clusters have equal sizes

In [50]:
# Apply K-means clustering to outputs
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(train_outputs.numpy())

# Create a balanced loader
balanced_train_loader = balanced_dataloader(trainset, clusters)

# Retrain the model using the balanced data
train_model(model, balanced_train_loader, optimizer, criterion)

Epoch 1, Loss: 0.0806948368534205
Epoch 2, Loss: 0.08339468362189982
Epoch 3, Loss: 0.08641058096082463
Epoch 4, Loss: 0.08311337330711208
Epoch 5, Loss: 0.07856441444604892


In [51]:
evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:12,  1.97it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:14,  1.54it/s]

Group (0, 1) Accuracy: 0.0


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:02<00:16,  1.35it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:12<00:00,  2.06it/s]

Group (4, 3) Accuracy: 2.0202020202020203


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:13<00:00,  1.87it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 0.0,
 (0, 2): 0.0,
 (0, 3): 0.0,
 (0, 4): 0.2364066193853428,
 (1, 0): 5.623471882640587,
 (1, 1): 100.0,
 (1, 2): 5.637254901960785,
 (1, 3): 0.0,
 (1, 4): 0.0,
 (2, 0): 3.7333333333333334,
 (2, 1): 0.5333333333333333,
 (2, 2): 100.0,
 (2, 3): 2.4,
 (2, 4): 0.0,
 (3, 0): 0.0,
 (3, 1): 0.0,
 (3, 2): 0.0,
 (3, 3): 100.0,
 (3, 4): 0.0,
 (4, 0): 21.91435768261965,
 (4, 1): 0.0,
 (4, 2): 0.0,
 (4, 3): 2.0202020202020203,
 (4, 4): 100.0}

In [52]:
evaluator.worst_group_accuracy

((0, 1), 0.0)

In [53]:
evaluator.average_accuracy

99.50674482729265

In [54]:
evaluator.evaluate_spurious_attribute_prediction()

97.6